In [2]:
from news_crawler import NewsCrawler
from IPython.display import display

In [3]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [4]:
# foxnews, foxbusiness를 합치자. 다른 소스도 집어넣자...

In [5]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (141.13 seconds): URLs collecting... investingcom        

n_total     10997
n_unique    10968
dtype: int64

huffpost               72
cnn                   811
investingcom          926
politico              217
time                   16
cnbc                  163
foxnews               189
foxbusiness           124
bbc                   285
businessinsider       582
morningstar           101
wsj                   100
nyt                    91
guardian              158
reuters              1162
washingtontimes       466
washingtonpost        134
cbs                   414
marketwatch           157
atlantic               98
vice                   19
npr                   847
newrepublic            20
yahoo                 233
independent           707
heritage              283
zdnet                 376
townhall              605
abcnews                95
hotair                 85
cbc                   293
nymag                 228
thestreet             149
thinkprogress          82
dailybeast             42
realclearpolitics     163
aljazeera             504
dtype: int64

,pubs
https://cbsnews.com/news/kanye-west-jesus-is-king-album-united-palace-kim-kardashian-secular-gospel-music,"cbs, realclearpolitics"
https://foxbusiness.com/lifestyle/president-jimmy-carter-birthday-95-money,"foxnews, foxbusiness"
https://foxbusiness.com/markets/houston-rockets-owner-business-mogul-tilman-fertitta-why-i-live-by-the-955-rule,"foxnews, foxbusiness"
https://foxbusiness.com/markets/stephen-schwarzman-tips-to-starting-a-business,"foxnews, foxbusiness"
https://foxbusiness.com/markets/warren-buffetts-duracell-sued-by-energizer,"foxnews, foxbusiness"
https://foxbusiness.com/money/bernie-sanders-income-inequality-tax-apple,"foxnews, foxbusiness"
https://foxbusiness.com/money/eric-trump-american-dream-alive-and-well,"foxnews, foxbusiness"
https://foxbusiness.com/money/government-mandated-wage-benefits-hurting-small-businesses,"foxnews, foxbusiness"
https://foxbusiness.com/money/heres-how-much-you-should-have-in-your-401k-to-reach-retirement-goals,"foxnews, foxbusiness"
https://foxbusiness.com/money/how-to-quit-stalling-and-write-your-will,"foxnews, foxbusiness"


# 2. 모은 URL중 이미 받은 것은 버린다

In [6]:
select_results = crawler.select(); display(*select_results)

100.00% (14.65 seconds): URLs selecting... aljazeera           

n_total     1651
n_unique    1646
dtype: int64

huffpost               4
cnn                   31
investingcom         121
politico              25
time                   1
cnbc                  19
foxnews               47
foxbusiness           21
bbc                   12
businessinsider      116
morningstar           10
wsj                   16
nyt                    4
guardian              25
reuters              232
washingtontimes       33
washingtonpost        31
cbs                   20
marketwatch           30
atlantic               2
vice                   0
npr                    1
newrepublic            0
yahoo                167
independent           31
heritage               0
zdnet                 10
townhall              28
abcnews               24
hotair                11
cbc                   23
nymag                 14
thestreet             24
thinkprogress          0
dailybeast             7
realclearpolitics      7
aljazeera            504
dtype: int64

,pubs
https://foxbusiness.com/lifestyle/president-jimmy-carter-birthday-95-money,"foxnews, foxbusiness"
https://foxbusiness.com/money/bernie-sanders-income-inequality-tax-apple,"foxnews, foxbusiness"
https://foxbusiness.com/money/trump-rips-fed-powell-own-worst-enemies,"foxnews, foxbusiness"
https://foxbusiness.com/real-estate/replacing-poles-with-pews-church-to-buy-former-strip-club-for-growing-congregation,"foxnews, foxbusiness"
https://foxbusiness.com/technology/leaked-mark-zuckerberg-audio-elizabeth-warren,"foxnews, foxbusiness"


# 3. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_results = crawler.download(); download_results # 뒤에 pub print는 빼도 되겠다...

55.78% (1037.05 seconds): downloading... businessinsider     

Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\Sejin\AppData\Local\Temp\jieba.cache


55.91% (1041.26 seconds): downloading... aljazeera           

Loading model cost 3.8068246841430664 seconds.
Prefix dict has been built succesfully.


78.07% (1847.56 seconds): downloading... aljazeera           

In [51]:
crawler.urls_final['thestreet']

{'downloaded': set(),
 'trashed': {'https://thestreet.com/partner/fight-over-hong-kong-lennon-wall-leaves-two-men-injured-as-tensions-escalate-between-anti-government-protesters-and-opponents-15108161',
  'https://thestreet.com/partner/people-s-bank-of-china-to-survey-30-000-households-about-debt-levels-as-financial-risks-continue-to-rise-15108163'}}